In [1]:
# import processing
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import tqdm
import librosa
import scipy
import torch
import torch.nn.functional as F

from scipy.spatial import distance
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from transformers import AutoProcessor, AutoModelForCTC
from phonemizer.backend.espeak.wrapper import EspeakWrapper

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

_ESPEAK_LIBRARY = r"C:\Program Files\eSpeak NG\libespeak-ng.dll"
EspeakWrapper.set_library(_ESPEAK_LIBRARY)
processor = AutoProcessor.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")
model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")




Some weights of the model checkpoint at facebook/wav2vec2-lv-60-espeak-cv-ft were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-lv-60-espeak-cv-ft and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably

![P1](https://dashpulsar.github.io/BrainlikeASR/Wav2vec2.png)

In [4]:
# Wav2vec2.0-Phoneme model structure
model

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projec

In [7]:
# Wav2vec2.0 encoder part model structure
model.wav2vec2

Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
      (1-4): 4 x Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=1024, bias=True)
    (dropout)

In [8]:
# CNNs layer structure
model.wav2vec2.feature_extractor

Wav2Vec2FeatureEncoder(
  (conv_layers): ModuleList(
    (0): Wav2Vec2LayerNormConvLayer(
      (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (activation): GELUActivation()
    )
    (1-4): 4 x Wav2Vec2LayerNormConvLayer(
      (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (activation): GELUActivation()
    )
    (5-6): 2 x Wav2Vec2LayerNormConvLayer(
      (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (activation): GELUActivation()
    )
  )
)

In [88]:
# for the 1st layer

random_input = torch.tensor(np.random.rand(1,16000)).float()

# change input size to 16005 by remove '#'of below code
#random_input = torch.tensor(np.random.rand(1,16005)).float()
output = model.wav2vec2.feature_extractor.conv_layers[0].conv(random_input)

"""
Why output is 512,3199?

Basicly it is depands on the parameter setting of the CNN layer. 
Output length = (input length - kernel size) / stride + 1
              = here (16000-10) /5 +1
              = 3199

you could simply get 3200 by change the input size to 16005

"""
print(output.shape)



torch.Size([512, 3199])


In [87]:
random_input = torch.tensor(np.random.rand(1,16000)).float()
output_CNN = model.wav2vec2.feature_extractor(random_input)
print(output_CNN.shape)

torch.Size([1, 512, 49])


In [99]:
model.wav2vec2.encoder.layers

ModuleList(
  (0-23): 24 x Wav2Vec2EncoderLayerStableLayerNorm(
    (attention): Wav2Vec2SdpaAttention(
      (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
      (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
      (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
      (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
    )
    (dropout): Dropout(p=0.1, inplace=False)
    (layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (feed_forward): Wav2Vec2FeedForward(
      (intermediate_dropout): Dropout(p=0.0, inplace=False)
      (intermediate_dense): Linear(in_features=1024, out_features=4096, bias=True)
      (intermediate_act_fn): GELUActivation()
      (output_dense): Linear(in_features=4096, out_features=1024, bias=True)
      (output_dropout): Dropout(p=0.1, inplace=False)
    )
    (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
)

In [ ]:
speech1="https://dashpulsar.github.io/BrainlikeASR/Wav_000_A_BOY_FELL_FROM_A_WINDOW.wav"

In [ ]:
speech=r"..\data\distance\ENG_ENG\ALL_055_M_ENG_ENG\Wav_000_A_BOY_FELL_FROM_A_WINDOW.wav"
speech_2=r"..\data\distance\ENG_ENG\ALL_055_M_ENG_ENG\Wav_014_HE_GREW_LOTS_OF_VEGETABLES.wav"